In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import pickle

Populating the interactive namespace from numpy and matplotlib


In [32]:
from dnn.data.dataloader import init_embed_loaders
from dnn.parser import get_sv_parser
from dnn.train.model import init_seed
from dnn.train.model import init_speechnet
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F

In [3]:
import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (4096, rlimit[1]))

In [4]:
def secToSample(sec):
    return int(16000 * sec)

In [8]:
options = get_sv_parser().parse_args(args=[])
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.data_folder = "/home/muncok/DL/dataset/SV_sets"
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768

In [20]:
options.input_format = "mfcc"
options.input_clip = True
options.input_length = secToSample(3) # if input_clip is false, it doesn't affect anything
options.splice_frames = secToSample(0.1)//160+1

In [10]:
options.data_folder = "/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/"

### SI_Model

In [23]:
options.input = "models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt"
options.model = "res15"
model = init_speechnet(options)
lda = None# None means not using lda

models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt is loaded


In [21]:
options.input = "models/reddots/simplecnn/si_reddots_0.2s_random_2.pt"
options.model = "SimpleCNN"
model =  init_speechnet(options)
lda = None

models/reddots/simplecnn/si_reddots_0.2s_random_2.pt is loaded


In [22]:
options.input = "models/commands/word_aligned.pt"
options.model = "SimpleCNN"
model = init_speechnet(options)
lda = pickle.load(open("models/lda/word_aligned_lda.pkl", "rb"))

models/commands/word_aligned.pt is loaded


###  Reddots Trial

In [24]:
import torch
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

def lda_on_tensor(tensor, lda):
    return torch.from_numpy(lda.transform(tensor.numpy()).astype(np.float32))

def embeds(opt, val_dataloader, model, lda=None):
    val_iter = iter(val_dataloader)
    model.eval()
    splice_dim = opt.splice_frames
    embeddings = []
    labels = []
    for batch in (val_iter):
        x, y = batch
        time_dim = x.size(2)
        split_points = range(0, time_dim-splice_dim+1, splice_dim)
        model_outputs = []
        for point in split_points:
            x_in = Variable(x.narrow(2, point, splice_dim))
            if opt.cuda:
                x_in = x_in.cuda()
            model_outputs.append(model.embed(x_in).cpu().data)
        model_output = torch.stack(model_outputs, dim=0)
        model_output = model_output.mean(0)
        if lda is not None:
            model_output = torch.from_numpy(lda.transform(model_output.numpy()).astype(np.float32))
        embeddings.append(model_output)
        labels.append(y.numpy())
    embeddings = torch.cat(embeddings)
    labels = np.hstack(labels)
    return embeddings, labels

In [25]:
m_part1_ndx = pd.read_pickle("trials/reddots/m_part1/m_part1_ndx.pkl")
m_part1_trn = pd.read_pickle('trials/reddots/m_part1/m_part1_trn.pkl')
# m_part4_ndx = pd.read_pickle("manifests/reddots/trial/m_part4/m_part4_ndx.pkl")
# m_part4_trn = pd.read_pickle("manifests/reddots/trial/m_part4/m_part4_trn.pkl")

In [26]:
trn = m_part1_trn
ndx = m_part1_ndx

In [27]:
err_type = {0:'TC', 1:'TW', 2:'IC', 3:'IW'}

###  Enrollment (trn)

In [39]:
val_dataloader = init_embed_loaders(options, trn) 
trn_embeddings, _ = embeds(options, val_dataloader, model, lda)

embed_dim = trn_embeddings.shape[-1]
trn_id = list(trn.id.unique())
spk_model_dict = {}
for id in trn_id:
    index = np.nonzero(trn.id == id)
    spk_model_dict[id] = trn_embeddings[index].mean(0, True)

###  SV Scoring (ndx)

In [31]:
m_part1_files = pd.read_pickle('trials/reddots/m_part1/m_part1_files.pkl')

In [29]:
val_dataloader = init_embed_loaders(options, m_part1_files) 
ndx_embeddings, _ = embeds(options, val_dataloader, model, lda)
# torch.save(ndx_embeddings, 'trials/reddots/m_part1/{}_embeds.pkl'.format(model_name))
# ndx_embeddings = torch.load('trials/reddots/m_part1/{}_embeds.pkl')

In [33]:
all_trials = ndx.id.unique().tolist()
scores = dict()
for t in range(4):
    scores[t] = []
    
for trial_id in tqdm(all_trials):
    trial_ndx = ndx[(ndx.id == trial_id)].reset_index()
    trial_embed_idx = np.nonzero(m_part1_files.file.isin(trial_ndx.file))
    trial_embeds = ndx_embeddings[trial_embed_idx]
    sim = F.cosine_similarity(trial_embeds, spk_model_dict[trial_id])
    for t in range(4):
        trial_type_idx = trial_ndx[trial_ndx.trial_type == t].index.tolist()
        scores[t].append(sim[trial_type_idx])
        
# [TC, TW, IC, IW]
for t in range(4):
    scores[t] = torch.cat(scores[t])

HBox(children=(IntProgress(value=0, max=320), HTML(value='')))

In [34]:
for t in range(4):
     print("{} mean:{:.2f}, std:{:.3f}".format(err_type[t], scores[t].mean(), scores[t].std()))

TC mean:1.00, std:0.000
TW mean:1.00, std:0.000
IC mean:1.00, std:0.000
IW mean:1.00, std:0.000


TD EERs

In [37]:
from sklearn.metrics import roc_curve


for t in range(1,4):
    score_vector = np.concatenate((scores[0], scores[t]))
    label_vector = np.concatenate((np.ones(len(scores[0])), 
                               np.zeros(len(scores[t]))))
    fpr, tpr, thres = roc_curve(label_vector, score_vector, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    thres = thres[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    print("[{}] eer: {:.2f}, thres: {:.5f}".format(err_type[t], eer, thres))

[TW] eer: 0.42, thres: 0.99989
[IC] eer: 0.31, thres: 0.99985
[IW] eer: 0.29, thres: 0.99984


TI EERs

In [38]:
from sklearn.metrics import roc_curve

score_vector = np.concatenate((scores[0], scores[1],
                              scores[2], scores[3]))
label_vector = np.concatenate((np.ones(len(scores[0]) + len(scores[1])), 
                           np.zeros(len(scores[2]) + len(scores[3]))))
fpr, tpr, thres = roc_curve(label_vector, score_vector, pos_label=1)
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
print("[TI] eer: {:.2f}".format(eer))

[TI] eer: 0.35
